In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.ndimage import uniform_filter1d  # simple smoothing

In [2]:
pd.set_option('display.max_columns', None)  # show all columns


Start with plot features

In [3]:
# get elevation features
plot_features = pd.read_pickle('data/plot_elev_features.pkl')
plot_features.head()

,key_0,plot_id,slope_mean,slope_min,slope_max,aspect_mean,aspect_min,aspect_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha
0,0,0,7.113986,1.849424,13.003842,168.632449,98.887299,283.569763,202.689667,227.668427,216.980925,17.724625,42.703384,32.015897,"POLYGON ((749515.552 5085017.582, 749593.102 5...",24.978760,37742.453705,3.774245
1,1,1,7.152989,2.414987,15.321275,169.715627,118.489319,219.622025,99.228783,120.792465,110.287463,-85.736259,-64.172577,-74.677583,"POLYGON ((749293.527 5084266.311, 749284.600 5...",21.563683,31915.687817,3.191569
2,2,2,11.799543,5.238199,19.049633,163.666617,144.183655,201.497757,105.721291,124.103233,115.271249,-79.243752,-60.861809,-69.693795,"POLYGON ((749683.152 5084433.659, 749663.238 5...",18.381943,10407.243140,1.040724
3,3,3,9.115794,2.130479,14.148518,165.906402,139.102295,192.083359,95.087128,109.772118,102.194042,-89.877914,-75.192924,-82.771002,"POLYGON ((749671.796 5084356.545, 749653.826 5...",14.684990,12707.534383,1.270753
4,4,4,6.510597,0.182567,14.545286,155.410061,77.834534,256.087616,130.938538,178.704865,152.918281,-54.026505,-6.260178,-32.046760,"POLYGON ((750026.827 5084777.385, 750031.798 5...",47.766327,127044.721505,12.704472


In [4]:
plot_ndvi = pd.read_pickle('data/plot_ndvi_filtered.pkl')

In [5]:
plot_ndvi

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,year,plot_id
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.151942,0.152856,0.153771,0.154686,0.155600,0.156515,0.157429,0.158344,0.159258,0.160173,0.161088,0.161849,0.162610,0.163372,0.164133,0.164895,0.165656,0.166418,0.167179,0.167941,0.168702,0.171981,0.175259,0.178538,0.178924,0.179309,0.179695,0.180081,0.180467,0.180852,0.181238,0.181624,0.182009,0.182395,0.182781,0.183167,0.183552,0.183938,0.184324,0.184710,0.185095,0.185481,0.185867,0.186253,0.186638,0.187024,0.187410,0.187796,0.188181,0.188567,0.188953,0.189339,0.189724,0.190110,0.190496,0.190881,0.191267,0.191653,0.192039,0.192274,0.192508,0.192743,0.192978,0.193213,0.193448,0.193682,0.193917,0.194152,0.194387,0.194622,0.194857,0.195091,0.195326,0.195561,0.195796,0.196031,0.196266,0.196500,0.196735,0.196970,0.197205,0.197440,0.197674,0.197909,0.198144,0.198379,0.198614,0.198849,0.199083,0.199318,0.199553,0.199788,0.200023,0.200258,0.200492,0.200727,0.200962,0.201197,0.201432,0.201666,0.201901,0.202136,0.202371,0.202606,0.202841,0.203075,0.203310,0.203545,0.203780,0.204015,0.204250,0.204484,0.204719,0.204954,0.209683,0.214411,0.218984,0.223557,0.228129,0.236301,0.244473,0.250892,0.257310,0.263729,0.272721,0.281713,0.287611,0.293509,0.299407,0.308094,0.316780,0.321200,0.325619,0.330039,0.336423,0.342807,0.346217,0.349627,0.353037,0.354432,0.355827,0.357223,0.358618,0.360013,0.361409,0.362804,0.363708,0.364612,0.365517,0.366421,0.367325,0.368229,0.369134,0.370038,0.373057,0.376077,0.377342,0.378607,0.379872,0.380669,0.381466,0.382262,0.383059,0.383855,0.383480,0.383104,0.383103,0.383102,0.383101,0.381989,0.380877,0.379205,0.377533,0.375861,0.373305,0.370749,0.369244,0.367739,0.366234,0.363878,0.361522,0.359391,0.357259,0.355128,0.353255,0.351381,0.350689,0.349997,0.349305,0.347807,0.346309,0.344230,0.342151,0.340072,0.337516,0.334960,0.334305,0.333650,0.332995,0.332339,0.331684,0.331029,0.330374,0.329719,0.326683,0.323646,0.322378,0.321109,0.319840,0.319548,0.319255,0.318963,0.318671,0.318378,0.318086,0.317794,0.317501,0.317209,0.316916,0.314631,0.312345,0.310674,0.309004,0.307333,0.306553,0.305772,0.304992,0.304212,0.303432,0.301157,0.298882,0.297378,0.295873,0.294368,0.293323,0.292278,0.291233,0.290188,0.289143,0.284933,0.280722,0.275346,0.269970,0.264594,0.264106,0.263618,0.263130,0.262641,0.262153,0.261665,0.261177,0.260689,0.260201,0.259713,0.259225,0.258736,0.258290,0.257843,0.257396,0.253737,0.250078,0.249505,0.248932,0.248359,0.247786,0.247213,0.246640,0.246067,0.245494,0.244921,0.244348,0.243775,0.243201,0.242628,0.241938,0.241247,0.240556,0.239865,0.239174,0.238483,0.237792,0.237102,0.236411,0.235720,0.235

In [6]:
doy = [i for i in range(0, 365, 1)]
grow_season = [i for i in doy if i >= 90 and i <= 300]

In [7]:
clean_rows = []

day_cols = list(range(365))

for idx, row in plot_ndvi.iterrows():
    ndvi = row[day_cols].values.astype(float)
    
    if np.all(np.isnan(ndvi)):
        continue
    
    # amplitude
    amp = np.nanmax(ndvi) - np.nanmin(ndvi)
    if amp < 0.05 or amp > 0.95:
        continue
    
    # smoothness
    diffs = np.abs(np.diff(ndvi))
    if np.nanmean(diffs) > 0.02:  # tune threshold
        continue
    
    # NaN coverage
    nan_frac = np.mean(np.isnan(ndvi))
    if nan_frac > 0.3:
        continue
    
    clean_rows.append(row)

ndvi_clean = pd.DataFrame(clean_rows)


In [8]:
# Clipped DOY range
start_doy = 90
end_doy = 300
day_cols = list(range(start_doy, end_doy + 1))

# Optional smoothing window (in days)
smooth_window = 15  # 5-day moving average

features = []

for _, row in ndvi_clean.iterrows():
    ndvi_raw = row[day_cols].values.astype(float)
    # Smooth NDVI to reduce noise
    ndvi = uniform_filter1d(ndvi_raw, size=smooth_window)

    year = row['year']
    plot_id = row['plot_id']

    # Peak and min
    peak_idx = np.nanargmax(ndvi)
    peak_val = ndvi[peak_idx]
    peak_doy = start_doy + peak_idx

    min_idx = np.nanargmin(ndvi)
    min_val = ndvi[min_idx]

    # Threshold-based SOS/EOS
    thresh = 0.2
    above_thresh = np.where(ndvi > thresh)[0]
    sos = start_doy + above_thresh[0] if len(above_thresh) > 0 else np.nan
    eos = start_doy + above_thresh[-1] if len(above_thresh) > 0 else np.nan
    los = eos - sos if not np.isnan(sos) and not np.isnan(eos) else np.nan

    # Integral (area under curve)
    integral = np.trapz(ndvi)

    # Slopes (avoid division by zero)
    greenup_slope = (peak_val - ndvi[above_thresh[0]]) / (peak_doy - sos) if len(above_thresh) > 0 and peak_doy != sos else np.nan
    senescence_slope = (ndvi[above_thresh[-1]] - peak_val) / (eos - peak_doy) if len(above_thresh) > 0 and eos != peak_doy else np.nan

    features.append({
        'year': year,
        'plot_id': plot_id,
        'ndvi_peak': peak_val,
        'ndvi_peak_doy': peak_doy,
        'ndvi_min': min_val,
        'ndvi_integral': integral,
        'ndvi_sos': sos,
        'ndvi_eos': eos,
        'ndvi_los': los,
        'ndvi_greenup_slope': greenup_slope,
        'ndvi_senescence_slope': senescence_slope
    })

ndvi_features = pd.DataFrame(features)


In [9]:
print(len(ndvi_features))
ndvi_features.isnull().sum()

363


year                     0
plot_id                  0
ndvi_peak                0
ndvi_peak_doy            0
ndvi_min                 0
ndvi_integral            0
ndvi_sos                 1
ndvi_eos                 1
ndvi_los                 1
ndvi_greenup_slope       1
ndvi_senescence_slope    3
dtype: int64

In [10]:
print('ndvi_features.describe()')
ndvi_features.describe()


ndvi_features.describe()


,year,plot_id,ndvi_peak,ndvi_peak_doy,ndvi_min,ndvi_integral,ndvi_sos,ndvi_eos,ndvi_los,ndvi_greenup_slope,ndvi_senescence_slope
count,363.000000,363.000000,363.000000,363.000000,363.000000,363.000000,362.000000,362.000000,362.000000,362.000000,360.000000
mean,2020.710744,31.561983,0.387498,211.415978,0.206513,65.804866,103.850829,297.320442,193.469613,0.001718,-0.001456
std,2.539537,19.186160,0.079139,34.790382,0.042964,12.331988,20.725080,14.195394,26.490377,0.000828,0.000737
min,2017.000000,0.000000,0.135585,90.000000,0.079908,21.274450,90.000000,113.000000,8.000000,0.000058,-0.005124
25%,2018.000000,15.000000,0.332397,184.000000,0.181090,57.429062,90.000000,300.000000,181.250000,0.001042,-0.001822
50%,2022.000000,31.000000,0.374060,197.000000,0.201515,64.864931,90.000000,300.000000,210.000000,0.001591,-0.001214
75%,2023.000000,49.000000,0.427289,239.000000,0.228637,73.291871,116.000000,300.000000,210.000000,0.002256,-0.000953
max,2024.000000,64.000000,0.672099,300.000000,0.351558,107.611319,292.000000,300.000000,210.000000,0.003952,-0.000044


In [11]:
ndvi_df = ndvi_features.dropna()

In [12]:
df = pd.merge(ndvi_df, plot_features, how = 'inner', on = 'plot_id')

In [13]:
weather = pd.read_pickle('data/PRISM/df.pkl')



weather = (
    weather
    .groupby("date", as_index=False)
    .first()
)


weather['date'] = pd.to_datetime(weather['date'])
weather['doy'] = weather['date'].dt.dayofyear
weather['year'] = weather['date'].dt.year

weather = weather[(weather['doy'] >= 90) & (weather['doy'] <= 300)]

weather

,date,n,ppt,tmax,tmean,tmin,vpdmax,vpdmin,doy,year
89,2016-03-30,1,0.0000,19.650000,10.914001,2.1790,17.035000,0.6790,90,2016
90,2016-03-31,1,0.0000,19.795000,11.609000,3.4230,15.902000,1.1440,91,2016
91,2016-04-01,1,0.0000,23.112000,13.090000,3.0690,19.868000,0.7460,92,2016
92,2016-04-02,1,0.0000,24.410000,14.909000,5.4100,19.346001,0.6970,93,2016
93,2016-04-03,1,0.0000,23.681000,15.196001,6.7120,20.033001,1.2670,94,2016
...,...,...,...,...,...,...,...,...,...,...
3217,2024-10-22,1,0.5341,18.301199,11.888399,5.4756,10.688600,0.4323,296,2024
3218,2024-10-23,1,0.0000,14.708099,9.087800,3.4675,8.405399,0.2547,297,2024
3219,2024-10-24,1,0.0000,15.271399,8.410000,1.5486,9.386300,0.2089,298,2024
3220,2024-10-25,1,0.0000,13.182099,6.662149,0.1422,8.109799,0.1778,299,2024


In [14]:
weather['date'] = pd.to_datetime(weather['date'])
weather['year'] = weather['date'].dt.year


In [15]:
# Compute frost days and GDD
weather['frost_days'] = (weather['tmin'] < 2).astype(int)
weather['gdd'] = (weather['tmean'] - 10).clip(lower=0)  # simple GDD with base 10°C

# Aggregate by year
weather_agg = weather.groupby('year').agg(
    ppt_sum=('ppt', 'sum'),
    ppt_min=('ppt', 'min'),
    ppt_max=('ppt','max'),
    ppt_avg=('ppt','mean'),
    ppt_var=('ppt','var'),
    tmean_max=('tmean','max'),
    tmean_min=('tmean', 'min'),
    tmean_avg=('tmean', 'mean'),
    tmean_var=('tmean','var'),
    tmax_max=('tmax','max'),
    tmax_min=('tmax', 'min'),
    tmax_mean=('tmax', 'mean'),
    tmax_var=('tmax','var'),
    tmin_max=('tmin','max'),
    tmin_min=('tmin', 'min'),
    tmin_avg=('tmin', 'mean'),
    tmin_var=('tmin','var'),

    vpdmax_mean=('vpdmax', 'mean'),
    vpdmax_max=('vpdmax','max'),
    vpdmax_min=('vpdmax','min'),
    vpdmax_var=('vpdmax','var'),

    vpdmin_mean=('vpdmin', 'mean'),
    vpdmin_max=('vpdmin','max'),
    vpdmin_min=('vpdmin','min'),
    vpdmin_var=('vpdmin','var'),    
    frost_days=('frost_days', 'sum'),
    gdd_sum=('gdd', 'sum')
).reset_index()


In [16]:
weather_agg

,year,ppt_sum,ppt_min,ppt_max,ppt_avg,ppt_var,tmean_max,tmean_min,tmean_avg,tmean_var,tmax_max,tmax_min,tmax_mean,tmax_var,tmin_max,tmin_min,tmin_avg,tmin_var,vpdmax_mean,vpdmax_max,vpdmax_min,vpdmax_var,vpdmin_mean,vpdmin_max,vpdmin_min,vpdmin_var,frost_days,gdd_sum
0,2016,82.668000,0.0,13.5400,0.391791,1.987807,28.642002,8.353001,18.471214,21.212821,38.674000,12.9130,26.077422,34.974402,18.965000,1.4650,10.866024,14.884056,25.786583,57.108002,2.4200,143.703666,2.367280,7.134000,0.1040,2.408359,1,1790.146190
1,2017,81.837000,0.0,13.2620,0.387853,3.188228,28.394001,7.240000,18.061323,37.356286,38.814999,10.7380,25.911834,56.035349,19.677000,-0.0070,10.211834,25.466729,26.484711,57.025002,3.1600,202.981751,2.220995,6.906000,0.0770,3.312897,12,1724.653181
2,2018,69.419000,0.0,12.1000,0.329000,1.717368,29.138000,5.372000,18.098124,30.557249,39.797001,11.6980,25.906616,46.483919,19.358999,-0.9520,10.290678,21.243658,26.500943,66.167999,1.3200,183.289876,2.152318,8.830000,0.0450,3.089984,7,1727.912173
3,2019,74.862999,0.0,8.4920,0.354801,1.368647,28.931000,5.086900,17.923297,29.345478,39.064999,9.4170,25.176616,42.744619,19.441000,-2.7730,10.670081,22.435182,24.029009,58.247002,1.5820,128.916470,1.628953,5.719000,0.0080,1.844163,7,1704.459209
4,2020,47.004000,0.0,12.2200,0.222768,1.330614,31.072498,-0.423000,18.232941,30.342420,41.867001,4.4500,25.841583,43.671245,20.870001,-5.2960,10.624393,23.786698,26.216488,68.960999,5.5230,154.665693,2.664706,9.945000,0.0120,3.712177,12,1791.087217
5,2021,48.150899,0.0,8.2375,0.228203,0.966819,34.620598,5.284300,18.770492,39.347299,46.087799,12.5773,26.701211,52.091722,23.825399,-2.2961,10.839911,31.984287,28.841146,90.093994,6.2786,223.740383,3.276940,11.210400,0.1313,5.604598,11,1875.636989
6,2022,120.400997,0.0,18.7108,0.570621,4.569819,31.220200,2.615000,17.990558,42.274508,42.470398,4.5380,25.586178,64.697810,21.471899,-1.3167,10.395076,27.847765,25.571178,71.424400,1.1144,225.269182,2.103550,8.014200,0.0319,4.198403,14,1753.124309
7,2023,54.430198,0.0,7.3282,0.257963,0.769340,30.042900,5.476100,18.569810,34.434660,40.726398,10.7560,26.189265,51.710022,20.337599,-1.1992,10.950490,24.070413,26.174509,64.660202,3.1125,197.708564,2.511833,9.505099,0.0490,5.621572,13,1862.293007
8,2024,42.795999,0.0,10.6019,0.202825,1.204203,30.093048,5.854000,17.999882,34.448677,41.072498,8.6102,25.944289,49.528720,22.065800,0.1025,10.055475,25.807756,26.204869,69.830498,2.4715,189.221817,2.588268,8.967299,0.0599,4.706999,8,1716.512808


In [17]:
df = pd.merge(df, weather_agg, how = 'inner', on = 'year')

In [18]:
df[df['ndvi_sos'] < 90]

,year,plot_id,ndvi_peak,ndvi_peak_doy,ndvi_min,ndvi_integral,ndvi_sos,ndvi_eos,ndvi_los,ndvi_greenup_slope,ndvi_senescence_slope,key_0,slope_mean,slope_min,slope_max,aspect_mean,aspect_min,aspect_max,elev_min,elev_max,elev_mean,elev_dev_min,elev_dev_max,elev_dev_mean,geometry,total_relief,area_m2,area_ha,ppt_sum,ppt_min,ppt_max,ppt_avg,ppt_var,tmean_max,tmean_min,tmean_avg,tmean_var,tmax_max,tmax_min,tmax_mean,tmax_var,tmin_max,tmin_min,tmin_avg,tmin_var,vpdmax_mean,vpdmax_max,vpdmax_min,vpdmax_var,vpdmin_mean,vpdmin_max,vpdmin_min,vpdmin_var,frost_days,gdd_sum


In [19]:
df.isna().sum()

year                     0
plot_id                  0
ndvi_peak                0
ndvi_peak_doy            0
ndvi_min                 0
ndvi_integral            0
ndvi_sos                 0
ndvi_eos                 0
ndvi_los                 0
ndvi_greenup_slope       0
ndvi_senescence_slope    0
key_0                    0
slope_mean               0
slope_min                0
slope_max                0
aspect_mean              0
aspect_min               0
aspect_max               0
elev_min                 0
elev_max                 0
elev_mean                0
elev_dev_min             0
elev_dev_max             0
elev_dev_mean            0
geometry                 0
total_relief             0
area_m2                  0
area_ha                  0
ppt_sum                  0
ppt_min                  0
ppt_max                  0
ppt_avg                  0
ppt_var                  0
tmean_max                0
tmean_min                0
tmean_avg                0
tmean_var                0
t

In [20]:
df.to_pickle('data/df.pkl')